In [ ]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Forecast as fc

In [ ]:
# loading data

provinces = pd.read_csv('..\data\dati-province\dpc-covid19-ita-province.csv')
today = '2021-06-26T17:00:00'
today_data = provinces.loc[provinces['data']==today]

population = pd.read_csv('..\data\Popolazione_province.csv', index_col=0)

p_list = pd.read_csv('..\data\Provinces_list.csv')['Sigla'].tolist()

x = today_data.loc[today_data['sigla_provincia'].isin(p_list)]['long'].values
y = today_data.loc[today_data['sigla_provincia'].isin(p_list)]['lat'].values
sigla = today_data.loc[today_data['sigla_provincia'].isin(p_list)]['sigla_provincia'].values
z = today_data.loc[today_data['sigla_provincia'].isin(p_list)]['totale_casi'].values

n_points = x.shape[0]

plt.figure()
for i in range(n_points):
    plt.scatter(x[i], y[i])
    plt.text(x[i], y[i], sigla[i])
plt.grid(True)



In [ ]:
# Extracting data for each province
df_list = []
for pr in p_list:
    df_list.append(provinces.loc[provinces['sigla_provincia']==pr])

n_days = df_list[0]['totale_casi'].values.shape[0]
new_pos_dict = {}

# index = p_list.index('BO')

for index, sigla in enumerate(p_list):
    plt.figure()
    plt.title(sigla)
    my_data = df_list[index]
    pos_tot = my_data['totale_casi'].values/population['Residenti'][sigla]*100000
    new_pos = pos_tot.copy()
    new_pos[1:] = (pos_tot[1:]-pos_tot[:-1])
    new_pos_dict.update({sigla: new_pos})
    plt.plot(new_pos, label=sigla)
    plt.grid(True)

In [ ]:
# Fitting data with Forecast class

waves_dict = {}

for index, sigla in enumerate(p_list):
    print('Fitting for: '+sigla)
    new_pos_tot = new_pos_dict[sigla]

    t0=14
    T=new_pos_tot.shape[0]
    tt_forecast = np.arange(t0,T)
    rt_list = []

    my_prediction = np.zeros(T-t0)

    Forecast = fc.n_waves(new_pos_tot,window=7)
    fit_data = np.zeros(T)
    for t in tt_forecast:
        fit_data[t] = Forecast.daily_update()
        rt_list.append(Forecast.rt)
        my_prediction[t-t0] = Forecast.predict(t+1)

    waves_dict.update({sigla: Forecast.L_waves})



In [ ]:
# Save the dictionary to a file using pickle
import pickle

save_flag=False
if save_flag:
    with open('..\data\provinces_waves.pickle', 'wb') as file:
        pickle.dump(waves_dict, file)